**Ingesting cleaned up production table and wbs project table for M7 analysis**

Creating two dataframes to store the data from both tables

In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql.functions import *
from pyspark.sql.types import *

prod_df = spark.sql("SELECT * FROM d365_prod.ProdTable")

proj_df = spark.sql("SELECT * FROM d365_prod.ProjPlanTable")

new_df = prod_df

StatementMeta(, 4aeeb0e4-928f-4fbc-9bdd-65380765a923, 4, Finished, Available, Finished)

Creating a new dataframe that joins the "Group", "Duration", "EffortInHours" columns from the wbsprojtable to the production table

This is done by comparing the project and subproject indicated between the two tables

In [3]:
new_df = new_df.join(proj_df, [new_df.Proj == proj_df.Proj, new_df.SubProj == proj_df.SubProj]).select(new_df["*"],proj_df["Group"],proj_df["Duration"],proj_df["EffortInHours"])

StatementMeta(, 4aeeb0e4-928f-4fbc-9bdd-65380765a923, 5, Finished, Available, Finished)

From the new dataframe the group column is used to indicate which M7 ECD is correct from the wbsproj table and is joined to the new dataframe

In [4]:
new_df = new_df.alias("A").join(proj_df.alias("B"), [new_df.Group == proj_df.Group, proj_df.Proj == "M7"],'left').select(col("A.*"),col("B.ECDDate"))

StatementMeta(, 4aeeb0e4-928f-4fbc-9bdd-65380765a923, 6, Finished, Available, Finished)

Changing the data types to match better to the data

In [5]:
new_df = new_df.withColumn("ECDDate",col("ECDDate").cast(DateType()))
new_df = new_df.withColumn("Created",col("Created").cast(DateType()))
new_df = new_df.withColumn("ImportDate",col("ImportDate").cast(DateType()))
new_df = new_df.withColumn('ReleaseDate',col('ReleaseDate').cast(DateType()))
new_df = new_df.withColumn('EstimateDate',col('EstimateDate').cast(DateType()))
new_df = new_df.withColumn('Status', regexp_replace('Status', '0','Created'))
new_df = new_df.withColumn('Status', regexp_replace('Status', '1','Estimated'))
new_df = new_df.withColumn('Status', regexp_replace('Status', '2','Scheduled'))
new_df = new_df.withColumn('Status', regexp_replace('Status', '3','Released'))
new_df = new_df.withColumn('Status', regexp_replace('Status', '4','Started'))
new_df = new_df.withColumn('Status', regexp_replace('Status', '5','Reported as finished'))
new_df = new_df.withColumn('Status', regexp_replace('Status', '7','Ended'))

StatementMeta(, 4aeeb0e4-928f-4fbc-9bdd-65380765a923, 7, Finished, Available, Finished)

In [6]:
new_df = new_df.sort(new_df['Production'].desc())
#display(new_df)

StatementMeta(, 4aeeb0e4-928f-4fbc-9bdd-65380765a923, 8, Finished, Available, Finished)

Create/overwrite a new table in datalake with the new dataframe

In [7]:
delta_table_path = "Tables/M7Table" #fill in your delta table path 
new_df.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(delta_table_path)

StatementMeta(, 4aeeb0e4-928f-4fbc-9bdd-65380765a923, 9, Finished, Available, Finished)